<font size=50 color=darkblue>Min Cut MILP</font>

# Problem modelling in LP format

## Import necessary modules
- Import function `read_model` from DOcplex to read the Min-Cut MILP from a temporary file
- Module `tempfile` is imported to create the temporary file
- Import function `remove` to delete the temporary file after reading it (required for python version < 3.12)

In [ ]:
from docplex.mp.model_reader import read_model
import tempfile
from os import remove

## Min-Cut MILP <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\sum_{(u,v)\in \mathcal{E}}\mu_{(u,v)}\cdot \color{blue}x_{(u,v)}$$
**Subject to**
### \begin{align*}
{\color{blue}z}_{\color{red}(t)} - {\color{blue}z}_{\color{green}(s)}&=1&\\
{\color{blue}x_{(u,v)}} &\ge {\color{blue}z_{(v)}} - {\color{blue}z_{(u)}},&\forall(u,v)\in \mathcal{E}\\
{\color{blue}x_{(u,v)}}&\in \left\{0,1\right\},&\forall(u,v)\in \mathcal{E}\\
{\color{blue}z_{(v)}} &\in \mathbb{R},&\forall v\in \mathcal{V}\\
\end{align*}

## Display the network
<img src='img/mc.png' width=700/>

## Write the Min-Cut MILP model (in LP format), which is assigned to the variable `mc_str`

In [ ]:
mc_str = '''
Minimize
 obj:   10 x(0,1) +  5 x(0,2) + 15 x(0,3) +  4 x(1,2) +  9 x(1,4)
      + 15 x(1,5) +  4 x(2,3) +  8 x(2,5) + 30 x(3,6) + 15 x(4,5)
      + 10 x(4,7) + 15 x(5,6) + 10 x(5,7) +  6 x(6,2) + 10 x(6,7)
Subject To
 c0: z(7) - z(0) = 1
 c1: x(0,1) + z(0) - z(1) >= 0
 c2: x(0,2) + z(0) - z(2) >= 0
 c3: x(0,3) + z(0) - z(3) >= 0
 c4: x(1,2) + z(1) - z(2) >= 0
 c5: x(1,4) + z(1) - z(4) >= 0
 c6: x(1,5) + z(1) - z(5) >= 0
 c7: x(2,3) + z(2) - z(3) >= 0
 c8: x(2,5) + z(2) - z(5) >= 0
 c9: x(3,6) + z(3) - z(6) >= 0
 c10: x(4,5) + z(4) - z(5) >= 0
 c11: x(4,7) + z(4) - z(7) >= 0
 c12: x(5,6) + z(5) - z(6) >= 0
 c13: x(5,7) + z(5) - z(7) >= 0
 c14: x(6,2) - z(2) + z(6) >= 0
 c15: x(6,7) + z(6) - z(7) >= 0

Binaries
 x(0,1) x(0,2) x(0,3) x(1,2) x(1,4) x(1,5) x(2,3) x(2,5) x(3,6) x(4,5) x(4,7)
 x(5,6) x(5,7) x(6,2) x(6,7)
End
'''

## To import the MILP model to DOcplex
- Store the model to a temporary file
- Have DOcplex read the model from the temporary file

In [ ]:
# Store string to tmp file
with tempfile.TemporaryFile(delete=False) as tmp:
    tmp.write(mc_str.encode('utf-8'))
    tmp.close()
    # Have DOcplex read the string
    mc_MILP = read_model(filename=tmp.name, model_name='Minimum Cut')
    # Delete the temporary file
    remove(tmp.name)

## Summarize the model

In [ ]:
mc_MILP.print_information()

## Solve the MILP and display the result

In [ ]:
mc_sol = mc_MILP.solve()
if mc_sol:
    mc_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`, `re`, `numpy`, `scipy`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy import interpolate
from scipy.spatial import ConvexHull

## Extract the node list `V`, link list `E`, link capacities $\mu$, source <code><font color='green'>s</font></code>, and sink <code><font color='red'>t</font></code> from the MILP model

In [ ]:
V, E, mu = [], [], {}

V = [int(re.findall('z\((.+)\)', var.name)[0]) for var in mc_MILP.iter_continuous_vars()]

for var in mc_MILP.iter_binary_vars():
    u, v = re.findall('x\((.+),(.+)\)', var.name)[0]
    E.append(e:=(int(u), int(v)))
    mu[e] = mc_MILP.objective_expr.get_coef(var)

for var, coef in mc_MILP.get_constraint_by_name('c0').get_left_expr().iter_terms():
    if coef == 1:
        t = int(re.findall(f'z\((.+)\)', var.name)[0])
    else:
        s = int(re.findall(f'z\((.+)\)', var.name)[0])

## For convenience, extract the solution to dictionaries named `sol_x` and `sol_z`

In [ ]:
sol_x = {(u,v): mc_sol.get_value(f'x({u},{v})') for u,v in E}
sol_z = {v: mc_sol.get_value(f'z({v})') for v in V}
print(f'{sol_x = }')
print(f'{sol_z = }')

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[s,t]['color'] = 'green', 'red'

g.es['label'] = [mu[e] for e in E]
g.es['arrow_size'] = [10 if sol_x[e] > 0 else 6 for e in E]
g.es['arrow_width'] = [10 if sol_x[e] > 0 else None for e in E]
g.es['width'] = [3 if sol_x[e] > 0 else 0.5 for e in E]
g.es['color'] = ['darkblue' if sol_x[e] > 0 else None for e in E]
g.es['label_size'] = 15

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

layout = g.layout('fr')
coords = np.asarray(layout.coords)
p = ig.plot(g, layout=layout, edge_label=g.es['label'], edge_background='white', target=ax)

bboxes = [path.get_extents().transformed(ax.transData.inverted()) for path in p.get_vertices().get_paths()]
radii = [max(bbox.height, bbox.width)/2 for bbox in bboxes]

S_circle_pads = [plt.Circle(coords[v], radii[v]*1.5) for v in V if sol_z[v] == sol_z[s]]
S_pts = np.asarray([pt for c in S_circle_pads for pt in c.get_patch_transform().transform(c.get_path().vertices)])

centroid = np.atleast_2d(S_pts[ConvexHull(S_pts).vertices].mean(0))

cross_pts = [(2/3)*coords[u, :] + (1/3)*coords[v, :] for u, v in E if sol_x[u, v] == 1]

hull = ConvexHull(bndry_pts:=np.vstack((S_pts, cross_pts)))
bndry_pts = bndry_pts[hull.vertices]
bndry_pts = np.unique(np.vstack((bndry_pts, cross_pts)), axis=0)
bndry_pts = bndry_pts[np.argsort(np.arctan2(*(bndry_pts - centroid).T))]

tck, u = interpolate.splprep(np.tile(bndry_pts, (20,1)).T, s=0)
out = interpolate.splev(np.linspace(0, 1, 1000), tck)

plt.fill(out[0], out[1], c='green', alpha=.3, ec=None)

plt.show()